<a href="https://colab.research.google.com/github/DanDas3/reconhecimento-digitos/blob/master/digit_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#coding: utf-8
import cv2 as cv
import numpy as np
import imutils
import os
import keras #Rede Neural
from keras.layers import MaxPooling2D, Flatten, Dense, Dropout
from matplotlib import pyplot as plt
from getpass import getpass
from os import listdir
from os.path import isfile, join

In [0]:
root = "reconhecimento-digitos"
data_root = "data"
out_path_root = "output"
digits_trains = "cvl-single-digits-train-validation/train"
digits_eval = "cvl-single-digits-train-validation/valid"
digits_valid = "cvl-single-digits-completeDatabase/cvl-single-digits/valid/"
string_trains = "cvl-strings-train/train"
string_eval = "cvl-strings-eval/cvl-strings-eval"

In [4]:
if os.path.exists(root) == False:
  !git clone https://github.com/DanDas3/reconhecimento-digitos.git

Cloning into 'reconhecimento-digitos'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 79500 (delta 21), reused 1 (delta 0), pack-reused 79445
Receiving objects: 100% (79500/79500), 187.79 MiB | 14.28 MiB/s, done.
Resolving deltas: 100% (25/25), done.
Checking out files: 100% (93529/93529), done.


In [0]:
# Funções

#def convertBin(img):
#  max_value = 255
#  neighborhood_size=99
#  subtract_from_mean = 10
#  return cv.adaptiveThreshold(img, max_value, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, neighborhood_size, subtract_from_mean)
  #return cv.threshold(img, 200, 255, cv.THRESH_BINARY)

def GaussianFilter(img):
  return cv.GaussianBlur(img, (5, 5), 0)

def morphologyFilter(img):
  img = cv.threshold(img, 225, 255, cv.THRESH_BINARY_INV)[1]
  img = cv.erode(img, cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 1)))
  img = cv.dilate(img, cv.getStructuringElement(cv.MORPH_ELLIPSE, (1, 3)))
  return img
def ordenaContornos(cnts):
  for i in range(len(cnts)):
      for j in range(i+1,len(cnts)):
          x_i = cv.boundingRect(cnts[i])[0]
          x_j = cv.boundingRect(cnts[j])[0]

          if(x_j < x_i):
              aux = cnts[j]
              cnts[j] = cnts[i]
              cnts[i] = aux
  return cnts  

def extrairRecortes(img, cnts):
  recorte = []
  copy = img.copy()
  for c in cnts:
    # dados do contorno
    (x, y, w, h) = cv.boundingRect(c)
    if w >= 5 and h >= 17:
      if (w >= 50):  # Divide a região em duas
        copy = cv.rectangle(copy, (x, y), (x + (w // 2), y + h), (0, 255, 0), 2)
        i = copy[y:y + h, x:x + (w // 2)]
        i = cv.resize(i, (32, 32))      
        recorte.append(i)

        i = copy[y:y + h, x + ((w // 2) + 1):x + w]
        copy = cv.rectangle(copy, (x, y), ((x + (w // 2) + 1) + (w // 2), y + h), (0, 255, 0), 2)
        i = cv.resize(i, (32, 32))
        # recorte.append(img[index_img][y:y + h, x + ((w // 2) + 1):x + w])
        recorte.append(i)
      else:
        copy = cv.rectangle(copy, (x, y), (x + w, y + h), (0, 255, 0), 2)
        i = copy[y:y + h, x:x + w]
        i = cv.resize(i, (32, 32))
        # recorte.append(img[index_img][y:y + h, x:x + w])
        recorte.append(i)
  return recorte, copy

def encontraContornos(img):
  cnts = cv.findContours(img.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  cnts = ordenaContornos(cnts)
  return cnts

def marcarSegmentos(img, positions):
  for position in positions:
    cv.rectangle(img, position[0],position[1],(0,0,0),thickness=2)
  return img

def carregaDigitTrain():
  path = root + "/" + data_root + "/" + digits_trains + "/"
  images = [f for f in listdir(path) if isfile(join(path, f))]
  data = np.ndarray((len(images), 32, 32, 3))
  tags = np.ndarray((len(images), 1))
  for i in range(len(images)):
    img = cv.imread(path + images[i])
    img = cv.resize(img,(32,32))
    tags[i] = int(images[i][:1])
    data[i] = img
  return data, tags

def carregaDigitEval():
  path = root + "/" + data_root + "/" + digits_eval + "/"
  images = [f for f in listdir(path) if isfile(join(path, f))]
  data = np.ndarray((len(images), 32, 32, 3))
  tags = np.ndarray((len(images), 1))
  for i in range(len(images)):
    img = cv.imread(path + images[i])
    img = cv.resize(img,(32,32))
    tags[i] = int(images[i][:1])
    data[i] = img
  
  return data, tags

def carregaDigitValid():
  path = root + "/" + data_root + "/" + digits_valid + "/"
  images = [f for f in listdir(path) if isfile(join(path, f))]
  data = np.ndarray((len(images), 32, 32, 3))
  tags = np.ndarray((len(images), 1))
  for i in range(len(images)):
    img = cv.imread(path + images[i])
    img = cv.resize(img,(32,32))
    tags[i] = int(images[i][:1])
    data[i] = img
  return data, tags

def carregaStringsValid():
  path = root + "/" + data_root + "/" + string_eval + "/"
  images = [f for f in listdir(path) if isfile(join(path, f))]
  #data = np.ndarray((len(images), 365, 32, 3))
  #tags = np.ndarray((len(images), 1))
  data = []
  digits = []
  for i in range(len(images)):
    img = cv.imread(path + images[i],0)
    #img = cv.resize(img,(365,89))    
    tag_num = images[i].split("-",1)[0]
    for j in range(len(tag_num)):
      digits.append(tag_num[j])
    data.append((img, digits))  
    #tags[i] = int(tag_num)
    #data[i] = img
  
  return data

In [0]:
data_train, tag_train = carregaDigitTrain()

data_eval, tag_eval = carregaDigitEval()

data_valid, tag_valid = carregaDigitValid()

string_digits=carregaStringsValid()

In [55]:
img = string_digits[0][0]

blur = GaussianFilter(img)

binImg = morphologyFilter(blur)

cnts = encontraContornos(binImg)

cnts = ordenaContornos(cnts)

marcado, recortes = marcarSegmentos(img, cnts)

#recortes = extrairRecortes(img, cnts)
#print(len(recortes))

fig,axes = plt.subplots(5, 1, figsize=(15, 5))
ax = axes.ravel()

ax[0].imshow(cv.cvtColor(img, cv.COLOR_GRAY2BGR))
ax[0].set_title("Imagem Original")
ax[0].set_axis_off()

ax[1].imshow(cv.cvtColor(blur, cv.COLOR_GRAY2BGR))
ax[1].set_title("Filtro Gaussiano")
ax[1].set_axis_off()

ax[2].imshow(binImg)
ax[2].set_title("Imagem Binarizada")
ax[2].set_axis_off()

ax[3].imshow(cv.cvtColor(marcado, cv.COLOR_BGR2RGB))
ax[3].set_title("Segmentos Marcados")
ax[3].set_axis_off()

#ax[4].imshow(cv.cvtColor(recortes[0], cv.COLOR_BGR2RGB))
ax[4].set_title("Recorte")
ax[4].set_axis_off()

SystemError: ignored

# Teste Rede Neural

In [0]:
#Rede Neural
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers

# Set random seed
np.random.seed(0)

# Set the number of features we want
number_of_features = 1000

# Load data and target vector from movie review data
(data_train, target_train), (data_test, target_test) = imdb.load_data(
num_words=number_of_features)

In [0]:
# Divide em treinamento, teste e validação
(data_train, data_valid) = data_train[number_of_features//2:], data_train[:number_of_features//2]
(target_train, target_valid) = target_train[number_of_features//2:], target_train[:number_of_features//2]
# Convert movie review data to one-hot encoded feature matrix
tokenizer = Tokenizer(num_words=number_of_features//2)
features_train = tokenizer.sequences_to_matrix(data_train, mode="binary")
features_valid = tokenizer.sequences_to_matrix(data_valid, mode="binary")

data_test = data_test[:500]
target_test = target_test[:500]
tokenizer = Tokenizer(num_words=number_of_features//2)
features_test = tokenizer.sequences_to_matrix(data_test, mode="binary")

In [0]:
# Start neural network
network = models.Sequential()

# Add fully connected layer with a ReLU activation function
network.add(layers.Dense(units=16, activation="relu", input_shape=(number_of_features//2,)))

# Add fully connected layer with a ReLU activation function
network.add(layers.Dense(units=16, activation="relu"))

# Add fully connected layer with a sigmoid activation function
network.add(layers.Dense(units=1, activation="sigmoid"))

In [0]:
# Compile neural network
network.compile(loss="binary_crossentropy", # Cross-entropy
optimizer="rmsprop", # Root Mean Square Propagation
metrics=["accuracy"]) # Accuracy performance metric

In [0]:
# Train neural network

from keras.callbacks import ModelCheckpoint   

# Salva o melhor resultado
checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1, save_best_only=True)

history = network.fit(features_train, # Features
target_train, # Target vector
epochs=20, # Number of epochs
verbose=2, # Print description after each epoch
batch_size=10, # Number of observations per batch
validation_data=(features_valid, target_valid), # Validation data
callbacks=[checkpointer],
shuffle=True)

Train on 24500 samples, validate on 500 samples
Epoch 1/20
 - 7s - loss: 0.3699 - acc: 0.8433 - val_loss: 0.3579 - val_acc: 0.8420

Epoch 00001: val_loss improved from inf to 0.35786, saving model to model.weights.best.hdf5
Epoch 2/20
 - 7s - loss: 0.3627 - acc: 0.8460 - val_loss: 0.3538 - val_acc: 0.8320

Epoch 00002: val_loss improved from 0.35786 to 0.35381, saving model to model.weights.best.hdf5
Epoch 3/20
 - 8s - loss: 0.3547 - acc: 0.8524 - val_loss: 0.3785 - val_acc: 0.8400

Epoch 00003: val_loss did not improve from 0.35381
Epoch 4/20
 - 8s - loss: 0.3470 - acc: 0.8577 - val_loss: 0.3731 - val_acc: 0.8340

Epoch 00004: val_loss did not improve from 0.35381
Epoch 5/20
 - 8s - loss: 0.3397 - acc: 0.8637 - val_loss: 0.4094 - val_acc: 0.8420

Epoch 00005: val_loss did not improve from 0.35381
Epoch 6/20
 - 8s - loss: 0.3330 - acc: 0.8682 - val_loss: 0.3988 - val_acc: 0.8240

Epoch 00006: val_loss did not improve from 0.35381
Epoch 7/20
 - 7s - loss: 0.3242 - acc: 0.8719 - val_loss

In [0]:
# Carrega os pesos com melho precisão na validação
network.load_weights('model.weights.best.hdf5')

In [0]:
print(features_test.shape)
print(features_test[0].shape)
print(features_test[0:1].shape)

(500, 500)
(500,)
(1, 500)


In [0]:
# Avaliaçãão do treinamento
score = network.evaluate(features_test, target_test, verbose=0)
print("\n","Resultado:", score[1])


 Resultado: 0.8400000009536743


In [0]:
y_hat = network.predict(features_test)

In [0]:
print(type(data_test[0]))
print(type(target_test[0]))
print(target_test[2])
for i, idx in enumerate(np.random.choice(features_test.shape[0], size=32, replace=False)):
  pred_idx = np.argmax(y_hat[idx])
  true_idx = np.argmax(data_test[idx])

  print(data_test[true_idx] == data_test[pred_idx] )

<class 'list'>
<class 'numpy.int64'>
1
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
